In [12]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math
import cv2


In [13]:
!git clone https://github.com/pytorch/vision.git
%cd vision
# !git checkout v0.8.2

!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../

%cd ..
!rm -r vision
!pip install pycocotools


Cloning into 'vision'...
remote: Enumerating objects: 337137, done.
remote: Counting objects: 100% (53369/53369), done.
remote: Compressing objects: 100% (1263/1263), done.
remote: Total 337137 (delta 52683), reused 52647 (delta 52078), pack-reused 283768
Receiving objects: 100% (337137/337137), 676.32 MiB | 27.11 MiB/s, done.
Resolving deltas: 100% (310534/310534), done.
/kaggle/working/vision
/kaggle/working


In [14]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    # return model
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    
    return model

In [15]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [16]:
def get_valid_data(labels):
    valid_data = []
    for i in range(len(labels)):
        for j in range(len(labels.iloc[i]['annotations'])):
            if labels.iloc[i]['annotations'][j]['type'] == "blood_vessel":
                valid_data.append(labels.iloc[i]['id'])
                break
    
    return valid_data

def gen_mask(annotations):
    boxes = []
    masks = []
    # area = []
    for i in range(len(annotations)):

        if annotations[i]['type'] == "blood_vessel":
            pts = np.array(annotations[i]['coordinates'])
            # print(pts[0][0])
            min_xy = np.min(pts, axis=1)[0]
            max_xy = np.max(pts, axis=1)[0]

            boxes += [np.concatenate((min_xy, max_xy), axis=0)]

            mask = np.zeros((512,512), dtype=np.uint8)
            cv2.fillPoly(mask, pts, 1)

            # area.append((max_xy[0] - min_xy[0]) * (max_xy[1] - min_xy[1]))

            masks += [mask]
        # break
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    masks = torch.as_tensor(masks, dtype=torch.uint8)
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

    return boxes , masks , area

In [17]:
class KidneyDataset(Dataset):
    def __init__(self , labels , metadata , image_list , tfm):
        self.labels = labels
        self.matadata = metadata
        self.image_list = image_list
        self.tfm = tfm
        
        self.image_with_target = {}

        image_path = Path(image_list[0]).parent
        print(image_path)
        
        for idx , col in labels.iterrows():
            
            # if f"{image_path}\\{col['id']}.tif" in image_list:
            if f"{image_path}/{col['id']}.tif" in image_list:
                polygons = col['annotations']
                # print(polygons[0])
                boxes , masks , area = gen_mask(polygons)
                iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
                
                target = {}
                target["boxes"] = boxes
                target["labels"] = torch.zeros((len(boxes),), dtype=torch.int64)
                target["masks"] = masks
                target["image_id"] = torch.tensor([idx])
                target["area"] = area
                target["iscrowd"] = iscrowd
                self.image_with_target[col['id']] = target

        # print()

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        
        # id = self.image_list[idx].split('\\')[-1].split('.')[0]
        id = self.image_list[idx].split('/')[-1].split('.')[0]
        img = Image.open(self.image_list[idx])
        target = self.image_with_target[id]


        if self.tfm is not None:
            img, target = self.tfm(img, target)
        # print(target)
        return img , target
        

In [18]:
ROOT = Path("/kaggle/input/hubmap-hacking-the-human-vasculature")
polygon_file = "polygons.jsonl"
image_folder = "train"
metadata_file = "tile_meta.csv"
train_valid_ratio = 0.9

labels = pd.read_json(ROOT / polygon_file , lines=True)
id_list = get_valid_data(labels)
metadata = pd.read_csv(ROOT / metadata_file)
image_list = [str(i) for i  in (ROOT / image_folder).glob('*.tif') if i.stem in id_list]
train_list = image_list[:int(len(image_list)*train_valid_ratio)]
valid_list = image_list[int(len(image_list)*train_valid_ratio):]

In [19]:

# train_set = KidneyDataset(labels , metadata , train_list , get_transform(train=True))

# # img , target = train_set[0]
# for i , j in train_set:
#     pass
#     break


In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = get_model_instance_segmentation(1).to(device)
batch_size = 2
n_epochs = 20
patience = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.0003, weight_decay=1e-5)
_exp_name = "mask-rcnn-model"
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size=3 , gamma=0.1)

In [21]:
import utils
train_set = KidneyDataset(labels , metadata , train_list , get_transform(train=True))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True , collate_fn=utils.collate_fn)
valid_set = KidneyDataset(labels , metadata , valid_list , get_transform(train=False))
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True , collate_fn=utils.collate_fn)
# valid_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
# trainset[0]


/kaggle/input/hubmap-hacking-the-human-vasculature/train
/kaggle/input/hubmap-hacking-the-human-vasculature/train


In [22]:
num_epochs = 10
from engine import train_one_epoch, evaluate

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader , device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    evaluate(model, valid_loader, device=device)
    torch.save(model.state_dict(), f"{_exp_name}_best.ckpt")

Epoch: [0]  [  0/730]  eta: 0:03:24  lr: 0.000001  loss: 0.9275 (0.9275)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 0.9047 (0.9047)  loss_rpn_box_reg: 0.0228 (0.0228)  time: 0.2799  data: 0.0473  max mem: 2012
Epoch: [0]  [ 10/730]  eta: 0:02:55  lr: 0.000005  loss: 1.0820 (1.4492)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 1.0431 (1.4043)  loss_rpn_box_reg: 0.0378 (0.0449)  time: 0.2432  data: 0.0249  max mem: 2018
Epoch: [0]  [ 20/730]  eta: 0:02:50  lr: 0.000009  loss: 1.2713 (1.6040)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 1.2405 (1.5486)  loss_rpn_box_reg: 0.0389 (0.0554)  time: 0.2379  data: 0.0214  max mem: 2018
Epoch: [0]  [ 30/730]  eta: 0:02:47  lr: 0.000013  loss: 1.6345 (1.6521)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000) 

KeyboardInterrupt: 

In [ ]:
# img = Image.open("test.jpg").convert("RGB")
# # img = tfm(img)
# model = get_model_instance_segmentation(2)
# model.eval()
 
# pred = model(img.unsqueeze(0))
# print(pred)

In [ ]:
# print(pred[0]['masks'].shape)
# Image.fromarray(pred[0]['masks'][0,0].mul(255).byte().cpu().numpy())